In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from googletrans import Translator
translator = Translator()
import re

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info


sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)

In [7]:
data = pd.read_pickle('data.pkl')
data

,date_block_num,shop_id,item_id,city_code,item_category_id,type_code,subtype_code,date_item_avg_item_cnt,date_avg_item_cnt,date_shop_avg_item_cnt,date_cat_avg_item_cnt,date_shop_cat_avg_item_cnt,date_shop_type_avg_item_cnt,date_shop_subtype_avg_item_cnt,date_city_avg_item_cnt,date_item_city_avg_item_cnt,date_type_avg_item_cnt,date_subtype_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,date_shop_revenue,shop_avg_revenue,month,days,item_cnt_month
0,0,2,19,0,40,8,14,0.022217,0.347168,0.141235,0.293945,0.073120,0.073120,0.073120,0.141235,0.0,0.293945,0.293945,28.000,28.0,1085064.75,1295577.625,0,31,0.0
1,0,2,27,0,19,6,32,0.155518,0.347168,0.141235,0.834961,0.485107,0.464844,0.511230,0.141235,1.0,0.925781,0.835449,1461.000,2324.0,1085064.75,1295577.625,0,31,1.0
2,0,2,28,0,30,6,30,0.177734,0.347168,0.141235,1.126953,0.537109,0.464844,0.505859,0.141235,0.0,0.925781,1.140625,310.000,549.0,1085064.75,1295577.625,0,31,0.0
3,0,2,29,0,23,6,43,0.088867,0.347168,0.141235,0.547363,0.208374,0.464844,0.251221,0.141235,0.0,0.925781,0.595215,1759.000,2398.0,1085064.75,1295577.625,0,31,0.0
4,0,2,32,0,40,8,14,5.378906,0.347168,0.141235,0.293945,0.073120,0.073120,0.073120,0.141235,0.0,0.293945,0.293945,249.625,338.0,1085064.75,1295577.625,0,31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,17,55,10,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,188.625,0.0,0.00,1203670.500,10,30,0.0
11128046,34,45,16188,17,64,7,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1257.000,0.0,0.00,1203670.500,10,30,0.0
11128047,34,45,15757,17,55,10,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,198.125,0.0,0.00,1203670.500,10,30,0.0
11128048,34,45,19648,17,40,8,14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,98.125,0.0,0.00,1203670.500,10,30,0.0


In [8]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [9]:
del data
gc.collect();

In [ ]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)